In [1]:
import numpy as np
import cv2, os
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, Dropout, MaxPool2D, AveragePooling2D, Conv2D, GlobalAvgPool2D, Flatten, BatchNormalization
from keras.layers.advanced_activations import ReLU
from keras.models import Model
np.random.seed(1337) # for reproducibility
from keras.optimizers import SGD
#from layers import ShakeShake
#from lr_callbacks import Step_decay, InternalStateHistory

Using TensorFlow backend.


In [2]:
batch_size = 64
nb_classes = 2
nb_epoch = 30
bn_axis = 3
actionDict = {}

In [3]:
def create_dict_of_actions():
    global actionDict
    actionDict = {}
    contents = []
    file = open("actions.txt", "r")
    i = 1
    if file.mode == 'r':
        fl = file.readlines()
        for f in fl:
            f = f.rstrip("\n")
            contents.append(f)
            actionDict[f] = i
            i += 1
            
        file.close()
        
create_dict_of_actions()
#print(actionDict)

{'applauding': 1, 'blowing_bubbles': 2, 'brushing_teeth': 3, 'cleaning_the_floor': 4, 'climbing': 5, 'cooking': 6, 'cutting_trees': 7, 'cutting_vegetables': 8, 'drinking': 9, 'feeding_a_horse': 10, 'fishing': 11, 'fixing_a_bike': 12, 'fixing_a_car': 13, 'gardening': 14, 'holding_an_umbrella': 15, 'jumping': 16, 'looking_through_a_microscope': 17, 'looking_through_a_telescope': 18, 'playing_guitar': 19, 'playing_violin': 20, 'pouring_liquid': 21, 'pushing_a_cart': 22, 'reading': 23, 'phoning': 24, 'riding_a_bike': 25, 'riding_a_horse': 26, 'rowing_a_boat': 27, 'running': 28, 'shooting_an_arrow': 29, 'smoking': 30, 'taking_photos': 31, 'texting_message': 32, 'throwing_frisby': 33, 'using_a_computer': 34, 'walking_the_dog': 35, 'washing_dishes': 36, 'watching_TV': 37, 'waving_hands': 38, 'writing_on_a_board': 39, 'writing_on_a_book': 40}


In [14]:
def paths_list_from_directory(directory):
    
    # Loading full image directories
    subdirs = [[] for i in range(2)]
    actions = [[] for i in range(40)]
    
    # Load all images
    for (dirpath,dirname,filename) in os.walk(directory):
        for file in filename:
            index = 0
            if file[:-8] in actionDict:
                index = actionDict[file[:-8]] - 1
            else: 
                continue
            actions[index].append(os.path.join(dirpath,file))
    
    # To have equal number of images for training, get action class with lowest number of images
    trainnum = 9999;
    for l in actions:
        if trainnum > len(l):
            trainnum = len(l)
    # Segment into training and testing 9:1
    trainnum = int(0.9*trainnum)
    
    # Segment each action class into training and testing 
    # using a dynamic test size so that equal number of
    # images are used per action for training
    for l in actions:
        test_size = (len(l) - trainnum) / len(l)
        action_train, action_test = train_test_split(l, test_size = test_size)
        subdirs[0].extend(action_train)
        subdirs[1].extend(action_test)
    #print(len(subdirs[0])," ",len(subdirs[1]))
    return subdirs

#paths = paths_list_from_directory('./JPEGImages')

6520   3012


In [ ]:
def load_image(filename):


    # [1] Get the file category and make the conversion. If 'dog' assign it integer 1, if 'cat' assign it integer 0.
    # Default label = cat
    label = 0
    if len(filename) == 0 or (not isinstance(filename, str)):
        # Error loading image
        label = -1
        return (None, -1)
    if filename.find('Dog') != -1:
        label = 1

    # [2] Load the image in greyscale with opencv.
    image = cv2.imread(filename,0)
    # Check that image read was successful
    if np.shape(image) != ():
            
        height, width = image.shape[:2]
        crop_dim = None
        
        # [3] Find the dimension that is the smallest between the height and the width and assign it to the crop_drim variable.
        crop_dim = min(height, width)
        
        # [4] Crop the centre of the image based on the crop_dim dimension for both the height and width.
        crop_img = image[(height-crop_dim)//2:(height+crop_dim)//2, (width-crop_dim)//2:(width+crop_dim)//2]
        
        # [5] Resize the image to 48 x 48 and divide it with 255.0 to normalise it to floating point format.
        image = cv2.resize(crop_img,(48,48))
        image = np.float32(image)
        image[:] = [x/255.0 for x in image]
        return (image,label)
    else:
        # Error loading image
        return (None, -1)

In [ ]:
def DataGenerator(img_addrs, img_labels, batch_size, num_classes):
    while 1:
        # Ensure randomisation per epoch
        addrs_labels = list(zip(img_addrs,img_labels))
        random.shuffle(addrs_labels)
        img_addrs, img_labels = zip(*addrs_labels)
        
        X = []
        Y = []
        for i in range(len(img_addrs)):
            # [1] Call the load_images function and 
            tpl = load_image(img_addrs[i])
            #     append the image in X.
            X.append(tpl[0])
            # [2] Create a one-hot encoding with np.eye and append the one-hot vector to Y.
            Y.append(np.eye(num_classes)[tpl[1]])
            # [3] Commpare the count and batch_size (hint: modulo operation) and if so:
            if len(X) % batch_size == 0 or (i+1) == len(img_addrs):
                # Reshape X to fit input for model
                lx = len(X)
                X = np.asarray(X)
                X = X.reshape(lx,2304)
                #   - Use yield to return X,Y as numpy arrays with types 'float32' and 'uint8' respectively  
                yield (np.float32(X),np.uint8(Y))
                #   - delete X,Y
                del X, Y
                #   - set X,Y to []
                X , Y = [],[]
                #   - use python garbage collector
                gc.collect()


In [ ]:
paths = paths_list_from_directory('./JPEGImages')

# Use train test split
X_train = []
Y_train = []
errs = []

for p in range(len(paths[0])):
    tpl = load_image(paths[0][p])
    if tpl[1] != -1:
        X_train.append(tpl[0])
        Y_train.append(np.eye(nb_classes)[np.uint8(tpl[1])])
    else:
        errs.append(p)
        
X_val = []
Y_val = []

for p in range(len(paths[1])):
    tpl = load_image(paths[1][p])
    if(tpl[1] != -1):
        X_val.append(tpl[0])
        Y_val.append(np.eye(nb_classes)[np.uint8(tpl[1])])

In [ ]:
lt = len(X_train)
lv = len(X_val)

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)

In [ ]:
# Should be changed to the right size
inputs = Input(shape=(2304,))

x = inputs
x = Conv2D(filters=32, kernel_size=(3, 3), name='cnn_lab_conv1')(x)
x = BatchNormalization(axis=bn_axis)(x)
x = Activation('relu')(x)

# Add MaxPool here

# Second block
x = Conv2D(filters=64, kernel_size=(3, 3), name='cnn_lab_conv2')(x)
x = BatchNormalization(axis=bn_axis)(x)
x = Activation('relu')(x)

# Add MaxPool here

# Third block
x = Conv2D(filters=128, kernel_size=(3, 3), name='cnn_lab_conv3')(x)
x = BatchNormalization(axis=bn_axis)(x)
x = Activation('relu')(x)

# x = ShakeShake()(x)

x = Flatten()(x)

In [ ]:
predictions = Dense(nb_classes, activation='softmax')(x)
model = Model(input=inputs, output=predictions)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
X_train = []

# Populate training set with image paths
for p in range(len(paths[0])):
    flag = 0
    for i in errs[0]:
        if (p==i):
            flag = 1
    if flag == 0:
        X_train.append(paths[0][p])

# Calculate step sizes for both sets
stepSizeTrain = math.ceil(len(X_train)/batch_size)
stepSizeVal = math.ceil(lv/batch_size)

#Convert labels to uint8
Y_train = np.uint8(Y_train)
Y_val = np.uint8(Y_val)

# Define generators
train_gen = DataGenerator(X_train, Y_train, batch_size = batch_size, num_classes = nb_classes)
#val_gen = DataGenerator(X_train, Y_train, batch_size = batch_size, num_classes = nb_classes)

# Run fit_generator
history = model.fit_generator(generator = train_gen,
                              epochs = nb_epoch,
                              steps_per_epoch = stepSizeTrain)

# Check score
score = model.evaluate(X_val, Y_val, verbose=0)
#history = model.fit_generator(
    #DataGenerator(X_val, Y_val, batch_size=batch_size, num_classes=nb_classes), 
    #steps_per_epoch = stepSize, epochs=nb_epoch)

print('Test score:', score[0])
print('Test accuracy:', score[1])
